/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT *
FROM Facilities
WHERE membercost != 0


/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT(*)
FROM Facilities
WHERE membercost =0

-- ANS:4

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost < .2 * monthlymaintenance
AND membercost >0


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
FROM Facilities
WHERE facid
IN (1,5)


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance,
    CASE WHEN monthlymaintenance >100 THEN 'expensive'
        ELSE 'cheap' END AS quality
FROM Facilities

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname
FROM Members
WHERE joindate = (
    SELECT MAX(joindate)
    FROM Members)

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT f.name, (firstname || ' ' || surname ) AS member_name
FROM Bookings AS b, Facilities AS f, Members AS m
WHERE (b.facid = f.facid)
    AND (b.memid = m.memid)
    AND f.name LIKE 'Tennis%'
    AND firstname != 'GUEST'
ORDER BY member_name


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT f.name, (firstname ||' ' || surname ) AS member_name,
    slots * (CASE WHEN b.memid =0 THEN guestcost
                ELSE membercost END) AS cost
FROM Bookings AS b, Facilities AS f, Members AS m
WHERE (b.facid = f.facid)
    AND (b.memid = m.memid)
    AND DATE( starttime ) = '2012-09-14'
    AND slots * (CASE WHEN b.memid =0 THEN guestcost
                    ELSE membercost END) > 30
ORDER BY cost DESC

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT * FROM (
    SELECT f.name, (firstname || ' ' || surname ) AS member_name,
        slots * (CASE WHEN b.memid =0 THEN guestcost
                ELSE membercost END) AS cost
    FROM Bookings AS b, Facilities AS f, Members AS m
    WHERE b.facid = f.facid
        AND b.memid = m.memid
        AND DATE( starttime ) = '2012-09-14') AS subq    
WHERE subq.cost > 30
ORDER BY cost DESC


/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  


In [15]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [6]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

In [8]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM Facilities
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [11]:
def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [12]:
#connect to database
sql_connect = sqlite3.connect('sqlite_db_pythonsqlite.db')

#The cursor() function is used to assist with executing our SQL queries
#It is important that you use the cursor() to return a Cursor instance corresponding to the database we want to query.
cursor = sql_connect.cursor()

### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [16]:
query = """
            select * 
            from(
                select subq.name, sum(subq.revenue) as total_revenue 
                from (
                    select f.name, case when b.memid = 0 then slots * guestcost
                                        else slots * membercost end as revenue
                    from Bookings as b, Facilities as f 
                    where b.facid = f.facid) as subq
                group by subq.name) as revsq
            where revsq.total_revenue < 1000
        """

#fetchall results --> saves as tuples
results = cursor.execute(query).fetchall()

#run query
pd.read_sql_query(query,sql_connect)

name  total_revenue
0     Pool Table            270
1  Snooker Table            240
2   Table Tennis            180

### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [18]:
query1 = """
            select m.surname as member_ln, m.firstname as member_fn, r.surname as rec_ln, r.firstname as rec_fn
            from Members as m, Members as r
            where m.recommendedby = r.memid and m.recommendedby != 0
            order by m.surname, m.firstname
        """

#fetchall results --> saves as tuples
results = cursor.execute(query1).fetchall()

#run query
pd.read_sql_query(query1,sql_connect)

member_ln  member_fn    rec_ln     rec_fn
0               Bader   Florence  Stibbons     Ponder
1               Baker       Anne  Stibbons     Ponder
2               Baker    Timothy   Farrell     Jemima
3              Boothe        Tim    Rownam        Tim
4             Butters     Gerald     Smith     Darren
5              Coplin       Joan     Baker    Timothy
6             Crumpet      Erica     Smith      Tracy
7                Dare      Nancy  Joplette     Janice
8             Genting    Matthew   Butters     Gerald
9                Hunt       John   Purview  Millicent
10              Jones      David  Joplette     Janice
11              Jones    Douglas     Jones      David
12           Joplette     Janice     Smith     Darren
13          Mackenzie       Anna     Smith     Darren
14               Owen    Charles     Smith     Darren
15             Pinker      David   Farrell     Jemima
16            Purview  Millicent     Smith      Tracy
17             Rumney  Henrietta   Genting    Matthew
18             Sarwin  Ramnaresh     Bader   Florence
19              Smith       Jack     Smith     Darren
20           Stibbons     Ponder     Tracy     Burton
21  Worthington-Smyth      Henry     Smith      Tracy

### Q12: Find the facilities with their usage by member, but not guests */

In [29]:
query2 = """
            select f.name as facility, m.firstname || ', ' || m.surname as member, sum(b.slots) as usage
            from Bookings b, Members m, Facilities f
            where b.facid = f.facid AND b.memid = m.memid
            AND b.memid != 0
            group by facility, member
            order by member, usage desc
        """

#fetchall results --> saves as tuples
results = cursor.execute(query2).fetchall()

#run query
pd.read_sql_query(query2,sql_connect)

facility           member  usage
0    Badminton Court  Anna, Mackenzie     96
1         Pool Table  Anna, Mackenzie     83
2       Table Tennis  Anna, Mackenzie     32
3      Snooker Table  Anna, Mackenzie     14
4       Squash Court  Anna, Mackenzie      4
..               ...              ...    ...
197       Pool Table     Tracy, Smith     64
198     Table Tennis     Tracy, Smith     56
199   Massage Room 1     Tracy, Smith     12
200     Squash Court     Tracy, Smith     12
201   Tennis Court 2     Tracy, Smith      6

[202 rows x 3 columns]

### Q13: Find the facilities usage by month, but not guests

In [39]:
query3 = """
            select f.name as facility, strftime('%m', starttime) as Month , sum(b.slots) as usage
            from Bookings b, Members m, Facilities f
            where b.facid = f.facid AND b.memid = m.memid
            AND b.memid != 0
            group by facility,month
            order by month, usage desc
        """

#fetchall results --> saves as tuples
results = cursor.execute(query3).fetchall()

#run query
pd.read_sql_query(query3,sql_connect)

facility Month  usage
0    Tennis Court 1    07    201
1    Massage Room 1    07    166
2   Badminton Court    07    165
3     Snooker Table    07    140
4    Tennis Court 2    07    123
5        Pool Table    07    110
6      Table Tennis    07     98
7      Squash Court    07     50
8    Massage Room 2    07      8
9   Badminton Court    08    414
10   Tennis Court 2    08    345
11   Tennis Court 1    08    339
12   Massage Room 1    08    316
13    Snooker Table    08    316
14       Pool Table    08    303
15     Table Tennis    08    296
16     Squash Court    08    184
17   Massage Room 2    08     18
18  Badminton Court    09    507
19       Pool Table    09    443
20   Tennis Court 1    09    417
21   Tennis Court 2    09    414
22    Snooker Table    09    404
23   Massage Room 1    09    402
24     Table Tennis    09    400
25     Squash Court    09    184
26   Massage Room 2    09     28